In [0]:
# Célula 1: Preparação dos Vetores
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# 1. Ler a tabela Silver
df = spark.read.table("portfolio_credit.german_credit_silver")

# 2. Definir quem são as colunas de entrada (Features)
feature_cols = [
    "duration", "credit_amount", "monthly_installment", "age",  # Numéricas
    "checking_status_ix", "credit_history_ix", "purpose_ix", "housing_ix" # Categóricas (Indexadas)
]

# 3. Criar o Vetor de Features
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_model = assembler.transform(df)

# Selecionar apenas o Vetor de Features e a Resposta (Label)
model_data = df_model.select("features", "label")

print("Dados vetorizados e prontos para o algoritmo.")
display(model_data.head(5))

In [0]:
# Célula 2: Split, Balanceamento e Treinamento (Versão Otimizada)
from pyspark.sql.functions import col, when

# 1. Divisão dos dados (70% Treino, 30% Teste)
train_data, test_data = model_data.randomSplit([0.7, 0.3], seed=42)

# --- TÉCNICA AVANÇADA: Balanceamento de Classes ---
# O problema: Temos muito mais "Bons" (0) do que "Ruins" (1).
# A solução: Vamos dar um "peso" maior para o erro na classe 1.

# Contamos quantos tem de cada no treino
dataset_size = train_data.count()
num_boms = train_data.filter(col("label") == 0).count()
num_ruins = train_data.filter(col("label") == 1).count()

# Calculamos o peso. Se temos 3x menos ruins, o peso deles será 3x maior.
balancing_ratio = num_boms / num_ruins
print(f"Ratio de Balanceamento: Cada erro de 'Mau Pagador' vale por {balancing_ratio:.2f} erros de 'Bom Pagador'.")

# Criamos a coluna de pesos no dataset de Treino
train_data_weighted = train_data.withColumn("classWeight", 
                                            when(col("label") == 1, balancing_ratio)
                                            .otherwise(1.0))

# 2. Criar e Treinar o Modelo (Random Forest Turbinado)
# weightCol: Usa o peso que calculamos
# numTrees: Aumentamos de 50 para 100 (Mais especialistas opinando)
# maxDepth: Aumentamos de 5 para 10 (Árvores mais profundas/inteligentes)
# subsamplingRate: Usa 80% dos dados para cada árvore (evita vício)

rf = RandomForestClassifier(
    labelCol="label", 
    featuresCol="features",
    weightCol="classWeight",  # <--- O Segredo do Balanceamento
    numTrees=100,             # Aumentamos
    maxDepth=10,              # Aumentamos
    subsamplingRate=0.8,
    seed=42
)

print("Treinando o modelo otimizado... (Pode demorar um pouquinho mais)")
rf_model = rf.fit(train_data_weighted)

print("Modelo treinado com sucesso!")

In [0]:
# Célula 3: Avaliação do Modelo

# 1. Fazer previsões nos dados de Teste (que o modelo nunca viu)
predictions = rf_model.transform(test_data)

# 2. Avaliar a performance
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)

print(f"--- RESULTADO DO MODELO ---")
print(f"AUC (Area Under Curve): {auc:.4f}")
print("-" * 30)

# Ver algumas previsões (prediction = 0 é Bom, 1 é Ruim)
display(predictions.select("label", "prediction", "probability").limit(10))

In [0]:
# Célula 4: Quais variáveis pesaram mais?
import pandas as pd

# Extrair a importância das variáveis do modelo treinado
importances = rf_model.featureImportances

# Criar um DataFrame simples para visualizar
feature_importance_df = pd.DataFrame(list(zip(feature_cols, importances)), columns=["Feature", "Importance"])
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)

# Mostrar gráfico
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.barplot(x="Importance", y="Feature", data=feature_importance_df, palette="viridis")
plt.title("Importância das Variáveis no Modelo de Crédito")
plt.show()

In [0]:
# Célula 5: Gerar Tabela Gold (Base completa com Scores)

# Aplicar modelo na base inteira (simulando score de produção)
full_predictions = rf_model.transform(df_model)

# Vamos explodir o vetor de probabilidade para pegar só a chance de ser "Ruim" (índice 1)
# Isso facilita muito para quem vai consumir a tabela depois
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import element_at

# Transforma a coluna vetorial 'probability' em array e pega o segundo elemento (Probabilidade de Default)
df_gold = full_predictions.withColumn("score_risco", element_at(vector_to_array("probability"), 2))
df_gold = df_gold.select("label", "prediction", "score_risco", *feature_cols) 

# Salvar
table_name = "portfolio_credit.german_credit_gold_predictions"
df_gold.write.format("delta").mode("overwrite").saveAsTable(table_name)

print(f"Tabela Gold com Scores salva: {table_name}")
display(df_gold)